In [1]:
import numpy as np
import pandas as pd
import re
import js2xml

from bs4 import BeautifulSoup
import requests

In [2]:
base_url = 'https://boardgamegeek.com/'
start_url = 'https://boardgamegeek.com/browse/boardgame/page/1'

In [3]:
test = requests.get(start_url).text

In [4]:
soup = BeautifulSoup(test,'lxml')

In [5]:
game_l = soup.find_all('td',{'class':'collection_objectname'})

In [590]:
page_n = 1
games = []
start_url = 'https://boardgamegeek.com/browse/boardgame/page/'
collect_list = ["objectid","name","yearpublished","sortindex","minplayers","maxplayers","minplaytime","maxplaytime",
                "minage","best","max","totalvotes",
                "playerage","languagedependence", "usersrated","average","baverage","stddev","avgweight", "numweights","numgeeklists",
                "numtrading","numwanting","numcomments","views","numplays","numplays_month","news",
                "blogs","weblink","podcast","label","boardgamedesigner","boardgameartist","boardgamepublisher",
               "boardgamehonor","boardgamecategory","boardgamemechanic","boardgameexpansion","boardgameversion",
               "boardgamefamily"]
credit_collect_list = ["boardgamedesigner","boardgameartist","boardgamepublisher","boardgamehonor","boardgamecategory",
                      "boardgameversion"]

while page_n <= 2 :
    game_names = BeautifulSoup(requests.get(start_url+str(page_n)).text,'lxml')
    game_list = soup.find_all('td',{'class':'collection_objectname'})
    if not game_list:
        print('No more results: Exiting...') #check to see if results exits, other wise break
        break
    else:
        print('Getting page:{}'.format(page_n))
        
        
        for game in game_list:
            game_data = []
            game_page = BeautifulSoup(requests.get(base_url+\
                        game.select('a')[0]['href']+'/stats').text,'html.parser')
            desc_tag = t_game.find_all('meta',{'name':'description'})[0]
            desc_str = str(desc_tag)[15:-29].replace('&amp;ldquo;','"').replace('&amp;rdquo;','"').replace('\n',' ')
            script =game_page.find("script", text=re.compile("GEEK.geekitemPreload\s+="))
            data_list = str(script).split("=",9)[9].strip().split(':{')
            comm_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[3][:-1].split(',') if len(i.split(':'))> 1 }
            stats_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[5][:-1].split(',') if len(i.split(':'))> 1 }
            counts_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[6][:-1].split(',') if len(i.split(':'))> 1 }
            info_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[7][:-1].split(',') if len(i.split(':'))> 1 }
            cr_cnt_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[9][:-1].split(',') if len(i.split(':'))> 1 }
            rank_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[10][:-1].split(',') if len(i.split(':'))> 1 }
            all_d = {**comm_d, **stats_d,**counts_d,**info_d,**cr_cnt_d,**rank_d}
            credit_d = {}
            for i in data_list[8].split('],'):
                item = i.split(":",1)
                try:
                    values = [h.split(',')[0] for h in [j.split(':')[1] for j in item[1].split('},{')]]
                except:
                    pass
                credit_d[item[0]]=values
            for i in collect_list:
                if '"'+i+'"' in all_d.keys():
                    game_data.append(all_d['"'+i+'"'].replace('"',''))
                else:
                    game_data.append('NaN')
            for i in credit_collect_list:
                if '"'+i+'"' in credit_d.keys():
                    game_data.append(credit_d['"'+i+'"'])
                else:
                    game_data.append('NaN')
            games.append(game_data)
        page_n += 1

Getting page:1
Getting page:2


In [591]:
df = pd.DataFrame.from_records(games)

In [592]:
df

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,174430,Gloomhaven,2017,1,1,4,60,120,12,[{min 3,...,12,4,19,7,"[""Isaac Childres""]","[""Alexandr Elichev"", ""Josh T. McDowell"", ""Alva...","[""Cephalofair Games"", ""Albi"", ""Asmodee"", ""Feue...","[""2017 Best Science Fiction or Fantasy Board G...","[""Adventure"", ""Exploration"", ""Fantasy"", ""Fight...","[""Chinese edition"", ""Czech edition"", ""English ..."
1,161936,Pandemic Legacy Season 1,2015,1,2,4,60,60,13,[{min 4,...,8,0,33,3,"[""Rob Daviau"", ""Matt Leacock""]","[""Chris Quilliams""]","[""Z-Man Games, ""Asterion Press"", ""Devir"", ""Fil...","[""2015 Cardboard Republic Immersionist Laurel ...","[""Environmental"", ""Medical""]","[""Chinese blue edition"", ""Chinese red edition""..."
2,230053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{min 3,...,9,15,29,6,NaN,NaN,NaN,NaN,NaN,NaN
3,196016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{min 3,...,5,1,14,2,NaN,NaN,NaN,NaN,NaN,NaN
4,224517,Brass Birmingham,2018,1,2,4,60,120,14,[{min 3,...,8,0,9,6,"[""Gavan Brown"", ""Matt Tolman"", ""Martin Wallace""]","[""Lina Cossette"", ""David Forest"", ""Damien Mamm...","[""Roxley"", ""BoardM Factory"", ""Conclave Editora...","[""2018 Golden Geek Best Board Game Artwork & P...","[""Economic"", ""Industry \/ Manufacturing"", ""Tra...","[""English deluxe edition"", ""English retail edi..."
5,233078,Twilight Imperium (Fourth Edition),2017,1,3,6,240,480,14,[{min 6,...,11,0,11,3,"[""Dane Beltrami"", ""Corey Konieczka"", ""Christia...","[""Scott Schomburg""]","[""Fantasy Flight Games"", ""ADC Blackfire Entert...","[""2017 Golden Geek Best Strategy Board Game No...","[""Civilization"", ""Economic"", ""Negotiation"", ""P...","[""Chinese edition"", ""Czech edition"", ""English ..."
6,292393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{min 2,...,9,9,32,5,NaN,NaN,NaN,NaN,NaN,NaN
7,187645,Star Wars Rebellion,2016,1,2,4,180,240,14,[{min 2,...,6,1,10,2,"[""Corey Konieczka""]","[""Matt Allsopp"", ""David Ardila"", ""Balaskas"", ""...","[""Fantasy Flight Games"", ""ADC Blackfire Entert...","[""2016 Best Science Fiction or Fantasy Board G...","[""Fighting"", ""Miniatures"", ""Movies \/ TV \/ Ra...","[""Czech edition"", ""English edition"", ""French e..."
8,220308,Gaia Project,2017,1,1,4,60,150,12,[{min 3,...,7,0,13,3,"[""Jens Dr\u00f6gem\u00fcller"", ""Helge Ostertag""]","[""Dennis Lohausen""]","[""Feuerland Spiele"", ""Cranio Creations"", ""Dice...","[""2017 Golden Geek Best Solo Board Game Nomine...","[""Civilization"", ""Economic"", ""Science Fiction""...","[""Chinese edition"", ""Dutch edition"", ""English ..."
9,169786,Scythe,2016,1,1,5,90,115,14,[{min 4,...,10,16,21,6,"[""Jamey Stegmaier""]","[""Jakub Rozalski""]","[""Stonemaier Games"", ""Albi"", ""Angry Lion Games...","[""2016 Cardboard Republic Architect Laurel Nom...","[""Economic"", ""Fighting"", ""Science Fiction"", ""T...","[""Chinese edition"", ""Czech edition"", ""English ..."


In [572]:
game_list[2]

<td class="collection_objectname" id="CEcell_objectname3">
<div id="status_objectname3"></div>
<div id="results_objectname3" onclick="" style="z-index:1000;">
<a href="/boardgame/167791/terraforming-mars">Terraforming Mars</a>
<span class="smallerfont dull">(2016)</span>
</div>
</td>

In [573]:
t_game = BeautifulSoup(requests.get(base_url+game_l[2].select('a')[0]['href']+'/stats').text,'html.parser')
title = t_game.title.text.split(' | ')[0]
desc_tag = t_game.find_all('meta',{'name':'description'})[0]
desc_str = str(desc_tag)[15:-29].replace('&amp;ldquo;','"').replace('&amp;rdquo;','"').replace('\n',' ')

In [596]:
t_game = BeautifulSoup(requests.get(base_url+game_l[2].select('a')[0]['href']+'/stats').text,'html.parser')
script =t_game.find("script", text=re.compile("GEEK.geekitemPreload\s+="))


In [597]:
data_list = str(script).split("=",9)[9].strip().split(':{')
comm_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[3][:-1].split(',') if len(i.split(':'))> 1 }
stats_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[5][:-1].split(',') if len(i.split(':'))> 1 }
counts_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[6][:-1].split(',') if len(i.split(':'))> 1 }
info_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[7][:-1].split(',') if len(i.split(':'))> 1 }
credit_d = {}
for i in data_list[8].split('],'):
    item = i.split(":",1)
    try:
        values = [h.split(',')[0] for h in [j.split(':')[1] for j in item[1].split('},{')]]
    except:
        pass
    credit_d[item[0]]=values
cr_cnt_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[9][:-1].split(',') if len(i.split(':'))> 1 }
rank_d = {i.split(':')[0]:' '.join(i.split(':')[1:]) for i in data_list[10][:-1].split(',') if len(i.split(':'))> 1 }
all_d = {**comm_d, **stats_d,**counts_d,**info_d,**cr_cnt_d,**rank_d}

In [598]:
all_d

{'"best"': '[{"min" 3',
 '"max"': '4}]',
 '"recommended"': '[{"min" 1',
 '"totalvotes"': '"1246"}',
 '"playerage"': '"12+"',
 '"languagedependence"': '"Moderate in-game text - needs crib sheet or paste ups"',
 '"subdomain"': '""',
 '"usersrated"': '"48046"',
 '"average"': '"8.42143"',
 '"baverage"': '"8.26713"',
 '"stddev"': '"1.36946"',
 '"avgweight"': '"3.2329"',
 '"numweights"': '"1855"',
 '"numgeeklists"': '"5904"',
 '"numtrading"': '"280"',
 '"numwanting"': '"2062"',
 '"numwish"': '"14585"',
 '"numowned"': '"62667"',
 '"numprevowned"': '"1887"',
 '"numcomments"': '"7247"',
 '"numwishlistcomments"': '"1486"',
 '"numhasparts"': '"21"',
 '"numwantparts"': '"8"',
 '"views"': '"4708268"',
 '"playmonth"': '"2020-01"',
 '"numplays"': '"244080"',
 '"numplays_month"': '"2658"',
 '"numfans"': '4023}',
 '"news"': '14',
 '"blogs"': '1155',
 '"weblink"': '60',
 '"podcast"': '148}',
 '"link"': '"https \\/\\/apps.apple.com\\/us\\/app\\/terraforming-mars\\/id1353471030?uo=4&mt=8&at=10lazE"',
 '"a

In [589]:
str(script).split('=',9)[9]

' {"item":{"itemdata":[{"datatype":"geekitem_fielddata","fieldname":"name","title":"Primary Name","primaryname":true,"required":true,"unclickable":true,"fullcredits":true,"subtype":"boardgame","keyname":"name"},{"datatype":"geekitem_fielddata","fieldname":"alternatename","title":"Alternate Names","alternate":true,"unclickable":true,"fullcredits":true,"subtype":"boardgame","keyname":"alternatename"},{"datatype":"geekitem_fielddata","fieldname":"yearpublished","title":"Year Published","fullcredits":true,"subtype":"boardgame","keyname":"yearpublished"},{"datatype":"geekitem_fielddata","fieldname":"minplayers","title":"Minimum Players","subtype":"boardgame","keyname":"minplayers"},{"datatype":"geekitem_fielddata","fieldname":"maxplayers","title":"Maximum Players","subtype":"boardgame","keyname":"maxplayers"},{"datatype":"geekitem_fielddata","fieldname":"minplaytime","title":"Minimum Playing Time","createposttext":" minutes","posttext":" minutes","subtype":"boardgame","keyname":"minplaytime

In [537]:
collect_list = ["objectid","name","yearpublished","sortindex","minplayers","maxplayers","minplaytime","maxplaytime",
                "minage","best","max","totalvotes",
                "playerage","languagedependence", "usersrated","average","baverage","stddev","avgweight", "numweights","numgeeklists",
                "numtrading","numwanting","numcomments","views","numplays","numplays_month","news",
                "blogs","weblink","podcast","label","boardgamedesigner","boardgameartist","boardgamepublisher",
               "boardgamehonor","boardgamecategory","boardgamemechanic","boardgameexpansion","boardgameversion",
               "boardgamefamily"]
credit_collect_list = ["boardgamedesigner","boardgameartist","boardgamepublisher","boardgamehonor","boardgamecategory",
                      "boardgameversion"]

In [576]:
all_d['"'+collect_list[0]+'"'].replace('"','')

'174430'

In [577]:
all_d

{'"best"': '[{"min" 3',
 '"max"': '4}]',
 '"recommended"': '[{"min" 1',
 '"totalvotes"': '"825"}',
 '"playerage"': '"14+"',
 '"languagedependence"': '"Extensive use of text - massive conversion needed to be playable"',
 '"subdomain"': '""',
 '"usersrated"': '"31052"',
 '"average"': '"8.85232"',
 '"baverage"': '"8.58396"',
 '"stddev"': '"1.60088"',
 '"avgweight"': '"3.8067"',
 '"numweights"': '"1304"',
 '"numgeeklists"': '"3640"',
 '"numtrading"': '"310"',
 '"numwanting"': '"1365"',
 '"numwish"': '"13722"',
 '"numowned"': '"50536"',
 '"numprevowned"': '"1791"',
 '"numcomments"': '"5935"',
 '"numwishlistcomments"': '"1058"',
 '"numhasparts"': '"27"',
 '"numwantparts"': '"9"',
 '"views"': '"8912535"',
 '"playmonth"': '"2020-01"',
 '"numplays"': '"228697"',
 '"numplays_month"': '"1928"',
 '"numfans"': '5640}',
 '"news"': '7',
 '"blogs"': '468',
 '"weblink"': '31',
 '"podcast"': '139}',
 '"itemid"': '1304840',
 '"objecttype"': '"thing"',
 '"objectid"': '174430',
 '"label"': '"Board Game"',


In [ ]:
page_n = 1
books = []
while True:
    html_titles = requests.get(start_url+str(page_n)).text #get page 'i'
    titles = BeautifulSoup(html_titles, 'lxml') #soupify
    prod_list = titles.find_all('div',{"class":"list_item"}) #get list of titles
    if not prod_list:
        print('No more results: Exiting...') #check to see if results exits, other wise break
        break
    else:
        print('Getting page:{}'.format(page_n))
        
        page_links = [prod_list[i].select('a')[1]['href'] for i in range(len(prod_list))] #list of links
        for link in page_links: #iterate through links)
            data = []
            request = requests.get(base_url + link).text
            soup = BeautifulSoup(request, 'lxml')            
            data.append(link.split('/')[2]) #append job id
            data.append(soup.find_all('h2',{'class':'title'})[0].text) #append title
            data.append(soup.find_all('div',{'class':'product-description'})[0].text) #append description
            spec = [i.text[:-1] for i in soup.find_all('dt')] #generate list of specs
            res = [i.text for i in soup.find_all('dd')] #generate spec details
            d = {i:j for i,j in zip(spec,res)}
            for i in base_specs: #iterate through base_specs; if present add to data, else append 'Na'
                if i in d.keys():
                    data.append(d[i])
                else:
                    data.append('Na')
            
            books.append(data) #append book data to list of books       
    page_n += 1 #go to next page

In [93]:
text = '{"item":{"itemdata":[{"datatype":"geekitem_fielddata","fieldname":"name","title":"Primary Name","primaryname":true,"required":true,"unclickable":true,"fullcredits":true,"subtype":"boardgame","keyname":"name"},{"datatype":"geekitem_fielddata","fieldname":"alternatename","title":"Alternate Names","alternate":true,"unclickable":true,"fullcredits":true,"subtype":"boardgame","keyname":"alternatename"},{"datatype":"geekitem_fielddata","fieldname":"yearpublished","title":"Year Published","fullcredits":true,"subtype":"boardgame","keyname":"yearpublished"},{"datatype":"geekitem_fielddata","fieldname":"minplayers","title":"Minimum Players","subtype":"boardgame","keyname":"minplayers"},{"datatype":"geekitem_fielddata","fieldname":"maxplayers","title":"Maximum Players","subtype":"boardgame","keyname":"maxplayers"},{"datatype":"geekitem_fielddata","fieldname":"minplaytime","title":"Minimum Playing Time","createposttext":" minutes","posttext":" minutes","subtype":"boardgame","keyname":"minplaytime"},{"datatype":"geekitem_fielddata","fieldname":"maxplaytime","title":"Maximum Playing Time","createposttext":" minutes","posttext":" minutes","subtype":"boardgame","keyname":"maxplaytime"},{"datatype":"geekitem_fielddata","fieldname":"minage","title":"Mfg Suggested Ages","createtitle":"Minimum Age","posttext":" and up","subtype":"boardgame","keyname":"minage"},{"datatype":"geekitem_fielddata","fieldname":"override_rankable","title":"Override Rankable","table":"geekitem_items","options":[{"value":1,"title":"yes"},{"value":0,"title":"no"}],"adminonly":true,"subtype":"boardgame","keyname":"override_rankable"},{"datatype":"geekitem_fielddata","fieldname":"targetco_url","unclickable":true,"title":"Target Co Order Link","subtype":"boardgame","keyname":"targetco_url"},{"datatype":"geekitem_fielddata","fieldname":"walmart_id","unclickable":true,"title":"Walmart Item Id","nullable":true,"subtype":"boardgame","keyname":"walmart_id"},{"datatype":"geekitem_fielddata","fieldname":"instructional_videoid","unclickable":true,"title":"Promoted Instructional Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"instructional_videoid"},{"datatype":"geekitem_fielddata","fieldname":"summary_videoid","unclickable":true,"title":"Promoted Summary Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"summary_videoid"},{"datatype":"geekitem_fielddata","fieldname":"playthrough_videoid","unclickable":true,"title":"Promoted Playthrough Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"playthrough_videoid"},{"datatype":"geekitem_fielddata","fieldname":"focus_videoid","unclickable":true,"title":"Promoted In Focus Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"focus_videoid"},{"datatype":"geekitem_fielddata","fieldname":"bggstore_product","unclickable":true,"title":"Promoted BGG Store Product Name","nullable":true,"subtype":"boardgame","keyname":"bggstore_product"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgamedesigner","linktype":"boardgamedesigner","self_prefix":"src","title":"Designer","titlepl":"Designers","fullcredits":true,"keyname":"boardgamedesigner"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgameartist","linktype":"boardgameartist","self_prefix":"src","title":"Artist","titlepl":"Artists","fullcredits":true,"keyname":"boardgameartist"},{"datatype":"geekitem_linkdata","other_objecttype":"company","other_subtype":"boardgamepublisher","linktype":"boardgamepublisher","self_prefix":"src","title":"Publisher","titlepl":"Publishers","required":true,"fullcredits":true,"keyname":"boardgamepublisher"},{"datatype":"geekitem_linkdata","other_objecttype":"family","other_subtype":"boardgamehonor","lookup_subtype":"boardgamehonor","linktype":"boardgamehonor","self_prefix":"src","title":"Honors","keyname":"boardgamehonor"},{"datatype":"geekitem_linkdata","other_objecttype":"property","other_subtype":"boardgamecategory","lookup_subtype":"boardgamecategory","linktype":"boardgamecategory","self_prefix":"src","title":"Category","titlepl":"Categories","showall_ctrl":true,"fullcredits":true,"keyname":"boardgamecategory"},{"datatype":"geekitem_linkdata","other_objecttype":"property","other_subtype":"boardgamemechanic","lookup_subtype":"boardgamemechanic","linktype":"boardgamemechanic","self_prefix":"src","title":"Mechanism","titlepl":"Mechanisms","showall_ctrl":true,"fullcredits":true,"keyname":"boardgamemechanic"},{"datatype":"geekitem_linkdata","lookup_subtype":"boardgame","other_objecttype":"thing","other_subtype":"boardgameexpansion","linktype":"boardgameexpansion","self_prefix":"src","title":"Expansion","keyname":"boardgameexpansion"},{"datatype":"geekitem_linkdata","other_objecttype":"version","other_subtype":"boardgameversion","linktype":"boardgameversion","loadlinks":true,"self_prefix":"src","title":"Version","uneditable":true,"keyname":"boardgameversion"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgame","lookup_subtype":"boardgame","linktype":"boardgameexpansion","self_prefix":"dst","title":"Expands","keyname":"expandsboardgame"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgame","lookup_subtype":"boardgame","linktype":"boardgameintegration","correctioncomment":"Only for stand-alone games that integrate with other stand-alone games. <b>NOT<\\/b> for expansions.","self_prefix":"src","title":"Integrates With","keyname":"boardgameintegration"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgame","lookup_subtype":"boardgame","linktype":"boardgamecompilation","self_prefix":"dst","correctioncomment":"Items contained in this item (if this is a compilation, for example)","title":"Contains","keyname":"contains"},{"datatype":"geekitem_linkdata","lookup_subtype":"boardgame","other_objecttype":"thing","other_subtype":"boardgame","linktype":"boardgamecompilation","self_prefix":"src","title":"Contained in","keyname":"containedin"},{"datatype":"geekitem_linkdata","lookup_subtype":"boardgame","other_objecttype":"thing","other_subtype":"boardgame","linktype":"boardgameimplementation","self_prefix":"src","correctioncomment":"Add the \\"child\\" item(s) that reimplement this game","title":"Reimplemented By","keyname":"reimplementation"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgame","lookup_subtype":"boardgame","linktype":"boardgameimplementation","self_prefix":"dst","correctioncomment":"Add the \\"parent\\" item(s) for this game, if it reimplements a previous game","title":"Reimplements","keyname":"reimplements"},{"datatype":"geekitem_linkdata","other_objecttype":"family","other_subtype":"boardgamefamily","lookup_subtype":"boardgamefamily","linktype":"boardgamefamily","self_prefix":"src","fullcredits":true,"title":"Family","keyname":"boardgamefamily"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"videogamebg","lookup_subtype":"videogame","linktype":"videogamebg","self_prefix":"src","adminonly":true,"title":"Video Game Adaptation","titlepl":"Video Game Adaptations","keyname":"videogamebg"},{"datatype":"geekitem_linkdata","other_objecttype":"family","other_subtype":"boardgamesubdomain","lookup_subtype":"boardgamesubdomain","linktype":"boardgamesubdomain","self_prefix":"src","title":"Subdomain","showall_ctrl":true,"uneditable":true,"createposttext":"Enter the subdomain for this item.","keyname":"boardgamesubdomain"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgameaccessory","lookup_subtype":"boardgameaccessory","linktype":"boardgameaccessory","self_prefix":"src","title":"Accessory","titlepl":"Accessories","addnew":true,"keyname":"boardgameaccessory"},{"datatype":"geekitem_polldata","title":"User Suggested # of Players","polltype":"numplayers","keyname":"userplayers"},{"datatype":"geekitem_polldata","title":"User Suggested Ages","polltype":"playerage","keyname":"playerage"},{"datatype":"geekitem_polldata","title":"Language Dependence","polltype":"languagedependence","keyname":"languagedependence"},{"datatype":"geekitem_polldata","title":"Subdomain","polltype":"boardgamesubdomain","keyname":"subdomain"},{"datatype":"geekitem_polldata","title":"Weight","polltype":"boardgameweight","keyname":"boardgameweight"}],"relatedlinktypes":["boardgameaccessory","contains","containedin","reimplementation","reimplements","videogamebg","boardgameintegration"],"linkedforum_types":[{"title":"Board Game Forums","linkedforum_index":"boardgames","linkdata_index":"expandsboardgame","required_subtype":null},{"title":"Expansion Forums","linkedforum_index":"boardgameexpansions","linkdata_index":"boardgameexpansion","required_subtype":null},{"title":"Compilation Item Forums","linkedforum_index":"compilationitems","linkdata_index":"contains","required_subtype":"boardgamecompilation"},{"title":"Parent Item Forums","linkedforum_index":"reimplements","linkdata_index":"reimplements","required_subtype":"boardgameimplementation"},{"title":"Reimplementation Forums","linkedforum_index":"reimplementedby","linkdata_index":"reimplementation","required_subtype":null},{"title":"Integrates With","linkedforum_index":"boardgameintegration","linkdata_index":"boardgameintegration","required_subtype":"boardgameintegration"},{"title":"Video Game Forums","linkedforum_index":"videogamebgs","linkdata_index":"videogamebg","required_subtype":null}],"subtypename":"Board Game","rankinfo":[{"prettyname":"Board Game Rank","shortprettyname":"Overall Rank","veryshortprettyname":"Overall","subdomain":null,"rankobjecttype":"subtype","rankobjectid":1,"rank":"1","baverage":"8.58426"},{"prettyname":"Thematic Rank","shortprettyname":"Thematic Rank","veryshortprettyname":"Thematic","subdomain":"thematic","rankobjecttype":"family","rankobjectid":5496,"rank":"1","baverage":"8.58878"},{"prettyname":"Strategy Game Rank","shortprettyname":"Strategy Rank","veryshortprettyname":"Strategy ","subdomain":"strategygames","rankobjecttype":"family","rankobjectid":5497,"rank":"1","baverage":"8.56365"}],"polls":{"userplayers":{"best":[{"min":3,"max":3}],"recommended":[{"min":1,"max":4}],"totalvotes":"824"},"playerage":"14+","languagedependence":"Extensive use of text - massive conversion needed to be playable","subdomain":"","boardgameweight":{"averageweight":3.8073676132003071,"votes":"1303"}},"stats":{"usersrated":"31004","average":"8.85259","baverage":"8.58426","stddev":"1.60083","avgweight":"3.8074","numweights":"1303","numgeeklists":"3636","numtrading":"311","numwanting":"1362","numwish":"13710","numowned":"50469","numprevowned":"1787","numcomments":"5931","numwishlistcomments":"1058","numhasparts":"27","numwantparts":"8","views":"8908418","playmonth":"2020-01","numplays":"228337","numplays_month":"1732","numfans":5633},"relatedcounts":{"news":7,"blogs":465,"weblink":31,"podcast":139},"itemid":1304840,"objecttype":"thing","objectid":174430,"label":"Board Game","labelpl":"Board Games","href":"\\/boardgame\\/174430\\/gloomhaven","subtype":"boardgame","subtypes":["boardgame","boardgameintegration"],"versioninfo":{"kickstarter_widget_url":"","gamepageorderurl":null,"shopifyitem":null},"name":"Gloomhaven","alternatename":null,"yearpublished":"2017","minplayers":"1","maxplayers":"4","minplaytime":"60","maxplaytime":"120","minage":"12","override_rankable":0,"targetco_url":"","walmart_id":"291875780","instructional_videoid":null,"summary_videoid":"102546","playthrough_videoid":null,"focus_videoid":null,"bggstore_product":"gloomhaven-solo-scenarios","links":{"boardgamedesigner":[{"name":"Isaac Childres","objecttype":"person","objectid":"69802","primarylink":0,"itemstate":"approved","href":"\\/boardgamedesigner\\/69802\\/isaac-childres"}],"boardgameartist":[{"name":"Alexandr Elichev","objecttype":"person","objectid":"77084","primarylink":0,"itemstate":"approved","href":"\\/boardgameartist\\/77084\\/alexandr-elichev"},{"name":"Josh T. McDowell","objecttype":"person","objectid":"78961","primarylink":0,"itemstate":"approved","href":"\\/boardgameartist\\/78961\\/josh-t-mcdowell"},{"name":"Alvaro Nebot","objecttype":"person","objectid":"84269","primarylink":0,"itemstate":"approved","href":"\\/boardgameartist\\/84269\\/alvaro-nebot"}],"boardgamepublisher":[{"name":"Cephalofair Games","objecttype":"company","objectid":"27425","primarylink":1,"itemstate":"approved","href":"\\/boardgamepublisher\\/27425\\/cephalofair-games"},{"name":"Albi","objecttype":"company","objectid":"4304","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/4304\\/albi"},{"name":"Asmodee","objecttype":"company","objectid":"157","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/157\\/asmodee"},{"name":"Feuerland Spiele","objecttype":"company","objectid":"22380","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/22380\\/feuerland-spiele"},{"name":"Games Warehouse","objecttype":"company","objectid":"40478","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/40478\\/games-warehouse"},{"name":"G\\u00e9m Klub Kft.","objecttype":"company","objectid":"8820","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/8820\\/gem-klub-kft"}],"boardgamehonor":[{"name":"2017 Best Science Fiction or Fantasy Board Game Nominee","objecttype":"family","objectid":"45000","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/45000\\/2017-best-science-fiction-or-fantasy-board-game-no"},{"name":"2017 Cardboard Republic Striker Laurel Nominee","objecttype":"family","objectid":"46988","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/46988\\/2017-cardboard-republic-striker-laurel-nominee"},{"name":"2017 Diana Jones Award for Excellence in Gaming Nominee","objecttype":"family","objectid":"42165","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/42165\\/2017-diana-jones-award-excellence-gaming-nominee"},{"name":"2017 Golden Geek Best Board Game Artwork & Presentation Nominee","objecttype":"family","objectid":"47451","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/47451\\/2017-golden-geek-best-board-game-artwork-presentat"},{"name":"2017 Golden Geek Best Cooperative Game Nominee","objecttype":"family","objectid":"47466","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/47466\\/2017-golden-geek-best-cooperative-game-nominee"},{"name":"2017 Golden Geek Best Cooperative Game Winner","objecttype":"family","objectid":"47423","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/47423\\/2017-golden-geek-best-cooperative-game-winner"}],"boardgamecategory":[{"name":"Adventure","objecttype":"property","objectid":"1022","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1022\\/adventure"},{"name":"Exploration","objecttype":"property","objectid":"1020","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1020\\/exploration"},{"name":"Fantasy","objecttype":"property","objectid":"1010","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1010\\/fantasy"},{"name":"Fighting","objecttype":"property","objectid":"1046","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1046\\/fighting"},{"name":"Miniatures","objecttype":"property","objectid":"1047","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1047\\/miniatures"}],"boardgamemechanic":[{"name":"Campaign \\/ Battle Card Driven","objecttype":"property","objectid":"2018","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2018\\/campaign-battle-card-driven"},{"name":"Cooperative Game","objecttype":"property","objectid":"2023","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2023\\/cooperative-game"},{"name":"Grid Movement","objecttype":"property","objectid":"2676","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2676\\/grid-movement"},{"name":"Hand Management","objecttype":"property","objectid":"2040","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2040\\/hand-management"},{"name":"Legacy Game","objecttype":"property","objectid":"2824","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2824\\/legacy-game"},{"name":"Modular Board","objecttype":"property","objectid":"2011","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2011\\/modular-board"}],"boardgameexpansion":[{"name":"Gloomhaven: Forgotten Circles","objecttype":"thing","objectid":"250337","primarylink":0,"itemstate":"approved","href":"\\/boardgame\\/250337\\/gloomhaven-forgotten-circles"},{"name":"Gloomhaven: Secrets of the Lost Cabal","objecttype":"thing","objectid":"297586","primarylink":0,"itemstate":"approved","href":"\\/boardgame\\/297586\\/gloomhaven-secrets-lost-cabal"},{"name":"Gloomhaven: Solo Scenarios","objecttype":"thing","objectid":"226868","primarylink":0,"itemstate":"approved","href":"\\/boardgame\\/226868\\/gloomhaven-solo-scenarios"},{"name":"Gloomhaven: The End of the World (Promo Scenario)","objecttype":"thing","objectid":"231934","primarylink":0,"itemstate":"approved","href":"\\/boardgame\\/231934\\/gloomhaven-end-world-promo-scenario"}],"boardgameversion":[{"name":"Chinese edition","objecttype":"version","objectid":"460243","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/460243\\/chinese-edition"},{"name":"Czech edition","objecttype":"version","objectid":"482203","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/482203\\/czech-edition"},{"name":"English edition second printing (with Solo Scenarios)","objecttype":"version","objectid":"384592","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/384592\\/english-edition-second-printing-solo-scenarios"},{"name":"English edition, fifth printing","objecttype":"version","objectid":"479734","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/479734\\/english-edition-fifth-printing"},{"name":"English edition, first printing","objecttype":"version","objectid":"331120","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/331120\\/english-edition-first-printing"},{"name":"English edition, fourth printing","objecttype":"version","objectid":"451774","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/451774\\/english-edition-fourth-printing"}],"expandsboardgame":[],"boardgameintegration":[{"name":"Gloomhaven: Jaws of the Lion","objecttype":"thing","objectid":"291457","primarylink":0,"itemstate":"approved","href":"\\/boardgame\\/291457\\/gloomhaven-jaws-lion"}],"contains":[],"containedin":[],"reimplementation":[],"reimplements":[],"boardgamefamily":[{"name":"Campaign Games","objecttype":"family","objectid":"24281","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/24281\\/campaign-games"},{"name":"Components: Miniatures","objecttype":"family","objectid":"25158","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/25158\\/components-miniatures"},{"name":"Crowdfunding: Kickstarter","objecttype":"family","objectid":"8374","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/8374\\/crowdfunding-kickstarter"},{"name":"Dungeon Crawler","objecttype":"family","objectid":"59218","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/59218\\/dungeon-crawler"},{"name":"Gloomhaven Universe","objecttype":"family","objectid":"45610","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/45610\\/gloomhaven-universe"},{"name":"Legacy","objecttype":"family","objectid":"25404","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/25404\\/legacy"}],"videogamebg":[{"name":"Gloomhaven","objecttype":"thing","objectid":"261209","primarylink":0,"itemstate":"approved","href":"\\/videogame\\/261209\\/gloomhaven"}],"boardgamesubdomain":[{"name":"Strategy Games","objecttype":"family","objectid":"5497","primarylink":0,"itemstate":"approved","href":"\\/boardgamesubdomain\\/5497\\/strategy-games"},{"name":"Thematic Games","objecttype":"family","objectid":"5496","primarylink":0,"itemstate":"approved","href":"\\/boardgamesubdomain\\/5496\\/thematic-games"}],"boardgameaccessory":[{"name":"Gloomhaven (second edition): Tower Rex Organizer","objecttype":"thing","objectid":"267166","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/267166\\/gloomhaven-second-edition-tower-rex-organizer"},{"name":"Gloomhaven: Adventure Tokens \\u2013 Altar","objecttype":"thing","objectid":"289347","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/289347\\/gloomhaven-adventure-tokens-altar"},{"name":"Gloomhaven: Adventure Tokens \\u2013 Bear Trap","objecttype":"thing","objectid":"289346","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/289346\\/gloomhaven-adventure-tokens-bear-trap"},{"name":"Gloomhaven: Adventure Tokens \\u2013 Bookcase","objecttype":"thing","objectid":"289348","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/289348\\/gloomhaven-adventure-tokens-bookcase"},{"name":"Gloomhaven: Adventure Tokens \\u2013 Boulder","objecttype":"thing","objectid":"289343","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/289343\\/gloomhaven-adventure-tokens-boulder"},{"name":"Gloomhaven: Adventure Tokens \\u2013 Cabinet","objecttype":"thing","objectid":"288415","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/288415\\/gloomhaven-adventure-tokens-cabinet"}]},"linkcounts":{"boardgamedesigner":1,"boardgameartist":3,"boardgamepublisher":9,"boardgamehonor":23,"boardgamecategory":5,"boardgamemechanic":12,"boardgameexpansion":4,"boardgameversion":19,"expandsboardgame":0,"boardgameintegration":1,"contains":0,"containedin":0,"reimplementation":0,"reimplements":0,"boardgamefamily":7,"videogamebg":1,"boardgamesubdomain":2,"boardgameaccessory":35},"secondarynamescount":0,"alternatenamescount":4,"primaryname":{"nameid":"972618","name":"Gloomhaven","sortindex":"1","primaryname":"1","translit":"Gloomhaven"}'